# 6.Pytorch

In [1]:
import torch
from torch import nn

## 1. 层绑定参数

nn.Sequential 类可以直接使用切片找到对应的层

如果在 nn.module 的类中没有使用self. 变量绑定模型，则无法将子层的参数用优化器优化，下方代码中的list 需要修改为 nn.Sequential 类。

```python
class MySequential(d2l.Module):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for idx, module in enumerate(args):
            self.modules.append(module)
            
    def forward(self, X):
        for module in self.modules:
            X = module(X)
        return X
    
ValueError: optimizer got an empty parameter list  # 报错
```

上述错误需要将模型挂在 self._modules下 , 该属性会初始化为有序字典（.state_dict()方法返回的就是该属性）。  
其中 key 与 value 均为 nn 的层，有序字典则模型层的顺序与传入的顺序相同。

In [4]:
class test(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, x):
        pass

net = test(nn.Linear(2,2), nn.ReLU())

In [11]:
type(list(net._modules.keys())[0])

torch.nn.modules.linear.Linear

## 2. 共享模型参数

In [12]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

/home/jovyan/.virtualenvs/basenv/lib/python3.7/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


由于模型参数包含梯度，因此在反向传播过程中，第二个共享隐藏层和第三个共享隐藏层的梯度会加在一起。

In [19]:
#  add_module 方法可以为模型添加具有名字的模块
net.add_module('test1', nn.Linear(2,2))

## 3. 参数初始化

自定义初始化参数，使用apply

In [53]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))
X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

注意使用带后缀_的函数， 这种类型表示替换而不是返回值。

In [54]:
def my_init(module):
    if type(module) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in module.named_parameters()][0])
        nn.init.uniform_(module.weight, -10, 10)
        module.weight.data *= module.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.4872,  0.0000, -0.0000, -0.0000],
        [-9.8747,  0.0000,  7.0784,  7.5458]], grad_fn=<SliceBackward>)

## 4. 自定义层

自定义层中需要在 self 中绑定相应的权重（参数类型）， 使用该类型的 data 进行计算。

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data  # 使用参数的 data 数据计算结果
        return F.relu(linear)

## 5. GPU device

在变量定义的时候直接指定device， 可以减少 to(device) 的时间，to 实际是在设备间 copy 数据。

<div style="text-align: center;">
  <img src="https://cdn.jsdelivr.net/gh/Huangl19/imgbed/202407301009797.png" width='1000px' >
</div>

如果不小心移动数据，可能会严重影响性能。  
一个典型的错误如下：在 GPU 上计算每个小批量的损失并在命令行上将其报告给用户（或将其记录在 NumPy 中ndarray）将触发全局解释器锁，从而使所有 GPU 停止运行。  
最好在 GPU 内部分配用于记录的内存并只移动较大的日志。

In [3]:
X = torch.ones(2, 3, device='cpu')  # 在定义变量的时候就指定设备